In [1]:
# default_exp models.classifiers

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from nbdev.export import *

In [12]:
# export
import torch
import torch.nn.functional as F
from fastcore.all import L, delegates, ifnone
from fvcore.common import registry
from timm.models.layers import create_classifier
from torch import nn
from torch.nn import Module

from src import _logger
from src.models.layers import *
from src.models.utils import *

CLASSIFIER_REGISTERY = registry.Registry("Classifiers")

In [13]:
import timm
import torchvision.transforms as T
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
from timm.data.transforms import RandomResizedCropAndInterpolation

from src.data.datasets import CassavaDataset, load_data

In [14]:
csv_path = "../../leaf-disease-classification-kaggle/data/stratified-data-5folds.csv"
imgs_dir = "../../Datasets/cassava/train_images/"

df = load_data(csv_path, imgs_dir)

t1 = T.Compose([
    RandomResizedCropAndInterpolation(512, interpolation="random"), 
    T.ToTensor(), 
    T.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)])

dset = CassavaDataset.from_torchvision_tfms(df, "filePath", label_col="label", transform=t1)
dl = torch.utils.data.DataLoader(dset, batch_size=4)

image, target = next(iter(dl))

In [15]:
# export
@CLASSIFIER_REGISTERY.register()
def CnnHeadV0(nf, n_out, pool_type="avg", use_conv=False, **kwargs):
    "create a classifier from timm lib"
    ls = create_classifier(nf, n_out, pool_type, use_conv)
    return nn.Sequential(*ls)

In [16]:
model = timm.create_model("tf_efficientnet_b3_ns", act_layer=ACTIVATIONS["mish"], drop_path_rate=0.25)
model = cut_model(model, -2)
feats = model(image)

In [18]:
head = CLASSIFIER_REGISTERY.get("CnnHeadV0")
head = head(nf=num_features_model(model), n_out=5, act_layer="default")
head

Sequential(
  (0): SelectAdaptivePool2d (pool_type=avg, flatten=True)
  (1): Linear(in_features=1536, out_features=5, bias=True)
)

In [19]:
logits = head(feats)
print(logits)

tensor([[-0.4737,  0.0571, -0.2030,  0.0642,  0.1819],
        [-0.1876, -0.0874, -0.1172,  0.2857,  0.1061],
        [ 0.1167, -0.0121, -0.0562,  0.0861, -0.1217],
        [-0.1057, -0.0203, -0.1438,  0.2405, -0.0189]],
       grad_fn=<AddmmBackward>)


Create a head for a model based on the `fast.ai` Library -

In [20]:
# export
@CLASSIFIER_REGISTERY.register()
def CnnHeadV1(nf, n_out, lin_ftrs=None, ps=0.5, concat_pool=True, 
              first_bn=True, lin_first=False, act_layer="default", **kwargs):
    "Model head that takes `nf` features, runs through `lin_ftrs`, and out `n_out` classes."
    if concat_pool:
        nf *= 2
    lin_ftrs = [nf, 512, n_out] if lin_ftrs is None else [
        nf] + lin_ftrs + [n_out]
    bns = [first_bn] + [True]*len(lin_ftrs[1:])
    ps = L(ps)

    if len(ps) == 1:
        ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps

    actns = [ACTIVATIONS[act_layer](inplace=True)] * (len(lin_ftrs)-2) + [None]

    pool = AdaptiveConcatPool2d() if concat_pool else nn.AdaptiveAvgPool2d(1)

    layers = [pool, nn.Flatten()]

    if lin_first:
        layers.append(nn.Dropout(ps.pop(0)))

    for ni, no, bn, p, actn in zip(lin_ftrs[:-1], lin_ftrs[1:], bns, ps, actns):
        layers += LinBnDrop(ni, no, bn=bn, p=p, act=actn, lin_first=lin_first)
    if lin_first:
        layers.append(nn.Linear(lin_ftrs[-2], n_out))
    return nn.Sequential(*layers)

In [22]:
head  = CLASSIFIER_REGISTERY.get('CnnHeadV1')(nf=num_features_model(model), n_out=5, act_layer="mish")

feats  = model(image[:2])
logits = head(feats)
print(logits)

tensor([[ 0.5345, -0.7971,  0.7221,  0.1647, -0.1956],
        [ 0.1962, -0.6849, -1.1612, -1.4432,  0.3658]], grad_fn=<MmBackward>)


In [24]:
# export
@CLASSIFIER_REGISTERY.register()
class CnnHeadV2(nn.Module):
    def __init__(self, nf, n_out, dropout=0.5, act_layer="mish", **kwargs):
        super().__init__()
        self.dropout = dropout
        self.act1 = ACTIVATIONS[act_layer](inplace=True)
        self.conv = nn.Conv2d(nf, nf, 1, 1)
        self.norm = nn.BatchNorm2d(nf)
        self.pool = GeM()

        self.fc1 = nn.Linear(nf, nf // 2)
        self.act2 = ACTIVATIONS[act_layer](inplace=True)
        self.rms_norm = RMSNorm(nf // 2)
        self.fc2 = nn.Linear(nf // 2, n_out)

    def forward(self, x):
        x = self.act1(x)
        x = self.conv(x)
        x = self.norm(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.dropout(x, p=self.dropout)
        x = self.fc1(x)

        x = self.act2(x)
        x = self.rms_norm(x)
        x = F.dropout(x, p=self.dropout / 2)
        x = self.fc2(x)
        return x

In [25]:
head  = CLASSIFIER_REGISTERY.get('CnnHeadV2')(nf=num_features_model(model), n_out=5, act_layer="mish")

feats  = model(image[:2])
logits = head(feats)
print(logits)

tensor([[-0.7211, -0.2441, -0.6659, -0.4313, -0.8709],
        [-0.4288,  0.3565,  0.3146,  0.1256, -0.1870]],
       grad_fn=<AddmmBackward>)


In [27]:
print(CLASSIFIER_REGISTERY)

Registry of Classifiers:
╒═══════════╤════════════════════════════════════════╕
│ Names     │ Objects                                │
╞═══════════╪════════════════════════════════════════╡
│ CnnHeadV0 │ <function CnnHeadV0 at 0x7ff675e8f3b0> │
├───────────┼────────────────────────────────────────┤
│ CnnHeadV1 │ <function CnnHeadV1 at 0x7ff67795ee60> │
├───────────┼────────────────────────────────────────┤
│ CnnHeadV2 │ <class '__main__.CnnHeadV2'>           │
╘═══════════╧════════════════════════════════════════╛


In [28]:
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.datasets.ipynb.
Converted 01a_data.mixmethods.ipynb.
Converted 02_losses.ipynb.
Converted 03_models.utils.ipynb.
Converted 03a_models.builder.ipynb.
Converted 03a_models.layers.ipynb.
Converted 03b_modules.classifiers.ipynb.
Converted 04_optimizers.ipynb.
Converted 04a_schedulers.ipynb.
Converted 05_lightning.data.ipynb.
Converted 05a_lightning.core.ipynb.
Converted 05b_lightning.callbacks.ipynb.
Converted 06_fastai.core.ipynb.
Converted index.ipynb.
